In [1]:
import sqlite3 as sq
import pandas as pd

Подключение к базе

In [2]:
with sq.connect('data/checking-logs.sqlite') as conn:
    conn.cursor()

Создание датасета test_results

1. **SQL-запрос начинается с ключевого слова SELECT** - которое указывает на выборку данных из таблицы.

2. **time, AVG(avg_diff) as avg_diff** - выбираются два столбца: time и среднее значение avg_diff. avg_diff - это среднее значение разницы между временем первого коммита (first_commit_ts) и сроком сдачи лабораторной работы (deadlines).

3. Затем следует подзапрос, который выполняет следующие действия:

    * **SELECT uid, labname, AVG ((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff** - выбираются столбцы uid, labname и вычисляется среднее значение avg_diff. 
    * **Функция strftime('%s', first_commit_ts)** - используется для преобразования даты и времени в формат временной метки в секундах.
    * **CASE WHEN (strftime('%s', first_commit_ts)) < (strftime('%s', first_view_ts)) THEN 'before' ELSE ' after' END AS time** - добавляется столбец time, который содержит значение "before", если время первого коммита (first_commit_ts) меньше времени первого просмотра (first_view_ts), и "after" в противном случае.
    * **FROM test LEFT JOIN deadlines ON test.labname = deadlines.labs** - указывается таблица test и производится соединение с таблицей deadlines по условию test.labname = deadlines.labs.
    * **WHERE labname != 'project1'** - добавляется условие, чтобы исключить лабораторную работу с именем "project1".
    * **GROUP BY uid, time** - выполняется группировка данных по uid и time.

4. Далее следует фильтрация данных с использованием следующего подзапроса:

    * **SELECT uid FROM (SELECT uid, CASE WHEN (strftime('%s', first_commit_ts)) < (strftime('%s', first_view_ts)) THEN 'before' ELSE ' after' END AS time FROM test LEFT JOIN deadlines ON test.labname = deadlines.labs WHERE labname != 'project1' GROUP BY uid, time) GROUP BY uid HAVING COUNT(uid) == 2** 
    - выбираются uid, у которых есть две записи с time "before" и "after". Здесь также применяется подзапрос для выборки uid и time, а затем выполняется группировка по uid и фильтрация с помощью HAVING COUNT(uid) == 2, чтобы выбрать только те uid, которые имеют две записи.

5. **Наконец, результаты группируются по time с помощью GROUP BY time.**

In [3]:
test_results = pd.io.sql.read_sql('''
                                SELECT time, AVG(avg_diff) as avg_diff
                                FROM (
                                SELECT uid, labname,
                                AVG ((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff,
                                CASE
                                WHEN (strftime('%s', first_commit_ts)) < (strftime('%s', first_view_ts))
                                THEN 'before' ELSE ' after' END AS time
                                FROM test
                                LEFT JOIN deadlines ON test.labname = deadlines.labs
                                WHERE labname != 'project1'
                                GROUP BY uid, time)
                                WHERE uid IN (
                                SELECT uid
                                FROM (
                                SELECT uid,
                                CASE
                                WHEN (strftime('%s', first_commit_ts)) < (strftime('%s', first_view_ts))
                                THEN 'before' ELSE ' after' END AS time
                                FROM test
                                LEFT JOIN deadlines ON test.labname = deadlines.labs
                                WHERE labname != 'project1'
                                GROUP BY uid, time)
                                GROUP BY uid
                                HAVING COUNT(uid) == 2)
                                GROUP BY time
                                ''', conn)
test_results

,time,avg_diff
0,after,-99.523810
1,before,-66.047619


Создание датасета control_results

In [4]:
control_results = pd.io.sql.read_sql('''
                                SELECT time, AVG(avg_diff) as avg_diff
                                FROM (
                                SELECT uid, labname,
                                AVG ((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff,
                                CASE
                                WHEN (strftime('%s', first_commit_ts)) < (strftime('%s', first_view_ts))
                                THEN 'before' ELSE ' after' END AS time
                                FROM control
                                LEFT JOIN deadlines ON control.labname = deadlines.labs
                                WHERE labname != 'project1'
                                GROUP BY uid, time)
                                WHERE uid IN (
                                SELECT uid
                                FROM (
                                SELECT uid,
                                CASE
                                WHEN (strftime('%s', first_commit_ts)) < (strftime('%s', first_view_ts))
                                THEN 'before' ELSE ' after' END AS time
                                FROM control
                                LEFT JOIN deadlines ON control.labname = deadlines.labs
                                WHERE labname != 'project1'
                                GROUP BY uid, time)
                                GROUP BY uid
                                HAVING COUNT(uid) == 2)
                                GROUP BY time
                                ''', conn)
control_results

,time,avg_diff
0,after,-99.322222
1,before,-98.033333


Закрытие соединения

In [5]:
conn.close()